In [1]:
!pip install markdown-it-py

  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)


In [2]:
!pip install PyGithub

  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 7.3 MB/s eta 0:00:00 MB/s eta 0:00:01
Using cached PyNaCl-1.5.0-cp36-abi3-macosx_10_10_universal2.whl (349 kB)
Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 47.4 MB/s eta 0:00:0031m53.2 MB/s eta 0:00:01


In [10]:
# Copy: https://github.com/Nordgaren/Github-Folder-Downloader/blob/master/gitdl.py
from github import Github, Repository, ContentFile
import requests
import os

def download(c: ContentFile, out: str):
    r = requests.get(c.download_url)
    output_path = f'{out}/{c.path}'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'wb') as f:
        print(f'downloading {c.path} to {out}')
        f.write(r.content)


def download_folder(repo: Repository, folder: str, out: str, recursive: bool):
    contents = repo.get_contents(folder)
    for c in contents:
        if c.download_url is None:
            if recursive:
                download_folder(repo, c.path, out, recursive)
            continue
        download(c, out)

In [ ]:
g = Github()
repo = g.get_repo("argilla-io/argilla-python")

In [11]:
download_folder(repo, "docs", "./argilla_sdk_docs", True)

downloading docs/assets/favicon.ico to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/argilla-slack.png to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/channels.PNG to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/compare-across-forks.PNG to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/compare-pull-request.PNG to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/create-branch-together.png to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/create-branch.PNG to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/create-fork.PNG to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/edit-file.PNG to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/fork-bar.PNG to ./argilla_sdk_docs
downloading docs/assets/images/community/contributing/issue-feature-template.PNG to ./arg

In [12]:
from pathlib import Path
from markdown_it import MarkdownIt

md = MarkdownIt("zero")


In [14]:
docs_path = Path("argilla_sdk_docs/docs")

In [22]:
md_files = list(docs_path.glob("**/*.md"))

In [28]:
def read_file(filename: Path) -> str:
    """Read a whole markdown file to a string."""
    with open(filename, "r") as f:
        return f.read()

In [29]:
contents = read_file(md_files[0])

In [31]:
md_tokens = md.parse(contents)

In [33]:
def is_front_matter(text: str) -> bool:
    """Check if a token pertains to the front matter.

    The check seeks if the string starts with '---' and
    the word `title` after a single line jump (it will fail if some
    space is inserted between them), and ends with '---'.

    Args:
        text (str):
            text obtained in the Token's content.
            Expects to be applied to the tokens from a markdown parsed.

    Returns:
        bool
    """
    return text.startswith("---\n") and text.endswith("\n---")


def is_figure(text: str) -> bool:
    """Check if a paragraph is just a picture in the doc.

    Some lines may contain just a picture, and there is no
    reason to translate those.
    i.e.
    '![helpner](/images/helpner-arch-part1.png)'
    The type of check is not perfect, it just fits my needs.

    Args:
        text (str): text obtained in the Token's content.

    Returns:
        bool:
    """
    text = text.strip()
    return text.startswith("![") and text.endswith(")")


def is_code(text: str) -> bool:
    """Check if a blob of text is a chunk of code.

    Args:
        text (str): text obtained in the Token's content.

    Returns:
        bool
    """
    text = text.strip()
    return text.startswith("```") and text.endswith("```")


def is_comment(text: str) -> bool:
    text = text.strip()
    return text.startswith("<!--") and text.endswith("-->")

In [484]:
def get_text_pieces(md_tokens):
    text_pieces = []
    for t in md_tokens:
        if t.type == "inline":
            if any(
                (
                    is_front_matter(t.content),
                    is_figure(t.content),
                    #is_code(t.content),
                    is_comment(t.content),
                )
            ):
                continue
            text_pieces.append(t.content)
    return text_pieces

In [519]:
#!pip install nltk

In [520]:
text_pieces

['# Installation',
 '## Install the SDK with pip',
 'Since this package is not yet published on PyPi, you can install it directly from the repository:',
 '```console\npip install git+https://github.com/argilla-io/argilla-python.git\n```',
 '## Run the Argilla server',
 'If you have already deployed Argilla Server, you can skip this step. Otherwise, you can quickly deploy it in two different ways:',
 '!!! note\n    You can use this SDK with any stable release of argilla server >= 1.27.',
 '* Using a [HF Space](https://huggingface.co/new-space?template=argilla/argilla-template-space).\n* Locally with Docker.',
 '```console\ndocker run -d --name quickstart -p 6900:6900 argilla/argilla-quickstart:latest\n```',
 '## Connect to the Argilla server',
 'Get your `<api_url>`:',
 '* If you are using HF Spaces, it should be constructed as follows: `https://[your-owner-name]-[your_space_name].hf.space`\n* If you are using Docker, it is the URL shown in your browser (by default `http://localhost:690

In [361]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/agus/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [481]:
from typing import List

def chunk_texts(text_pieces: List[str], max_chars: int = 512):
    chunks = []
    current_chunk = []
    current_chars = 0

    for text in text_pieces:
        if len(text) > max_chars:
            sentences = nltk.sent_tokenize(text)
            if len(sentences) > 1:
                # To avoid infinite recursion
                pieces = chunk_texts(sentences, max_chars)
                chunks.extend(pieces)
            else:
                current_chunk.extend(sentences)
                chunks.extend(sentences)

        elif current_chars + len(text) > max_chars:
            chunks.append(". ".join(current_chunk))
            current_chunk = [text]
            current_chars = len(text)
        
        else:
            current_chunk.append(text)
            current_chars += len(text)

    if current_chunk:
        chunks.append(". ".join(current_chunk))

    return chunks


In [482]:
max_chars = 256
chunks = chunk_texts(text_pieces, max_chars=max_chars)
[len(c) for c in chunks]
#chunks

[255,
 223,
 226,
 68,
 244,
 226,
 57,
 260,
 153,
 188,
 249,
 120,
 236,
 384,
 342,
 728,
 208,
 109,
 276,
 278,
 797,
 101,
 142]

In [487]:
#!pip install datasets

In [488]:
from datasets import Dataset

/Users/agus/github_repos/argilla-io/distilabel-workbench/projects/argilla-sdk-bot/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [493]:
max_chars = 512

data = {}

for file in md_files:
    contents = read_file(file)
    md_tokens = md.parse(contents)
    text_pieces = get_text_pieces(md_tokens)
    chunks = chunk_texts(text_pieces, max_chars=max_chars)
    data[str(file)] = chunks

In [511]:
data
df = pd.DataFrame.from_records([(k, v) for k, values in data.items() for v in values], columns=["filename", "chunks"])
ds = Dataset.from_pandas(df)

In [514]:
ds.push_to_hub("plaguss/argilla_sdk_docs_raw", private=False)

Uploading the dataset shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.07s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/plaguss/argilla_sdk_docs_raw/commit/5f16ab3b169a56f0765be0fad96e7145e74209a4', commit_message='Upload dataset', commit_description='', oid='5f16ab3b169a56f0765be0fad96e7145e74209a4', pr_url=None, pr_revision=None, pr_num=None)